In [16]:
import numpy as np
import tensorflow as tf
import kerastuner as kt
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline 
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import sklearn.metrics as metrics

In [20]:
df= pd.read_csv("all_data_LSTM_full_feature.csv")
test=pd.read_csv("y_DX_month.csv")
df=df.sort_values(by='RID', ascending=True)
test=test.sort_values(by='RID', ascending=True)
groupby=df.groupby("RID").count()

In [3]:
df=df[df.VISCODE2=='bl']

In [4]:
df=df.drop(['RID', 'VISCODE2'],axis = 1 )

In [5]:
num_features=len(df.columns)
y= np.array(test.DX_Month)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(df)
x = scaler.transform(df)
scaler2 = MinMaxScaler(feature_range=(0, 1))
y = scaler2.fit_transform(pd.DataFrame(y))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
     x, y, test_size=0.10, stratify= y, shuffle=True, random_state=None) 

In [7]:
def build_model(hp):
    num_units_min  =  200
    num_units_max  =  1000
    num_units_step =  50

    dropout_min  =  .2
    dropout_max  =  0.9
    dropout_step =  0.1
    
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('unit1',  min_value=num_units_min,
                                                 max_value=num_units_max,
                                                 step=num_units_step),
                                                 activation='relu',
                                                 input_dim=num_features, 
                                                 kernel_initializer='normal', 
                                                 kernel_regularizer=keras.regularizers.l2(hp.Choice('reg_rate',values=[0.01, 0.05, 0.1]))
                          
                                               ))
    model.add(layers.Dropout(hp.Float('dropout_1',min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step) ) )
 
 
    model.add(layers.Flatten())
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(units=hp.Int('unitsdense_'+ str(i),
                                            min_value=200,
                                            max_value=800,
                                            step=200),
                                            activation='relu',
                              kernel_regularizer=keras.regularizers.l2(hp.Choice('reg_rate1',values=[0.01, 0.05, 0.1]))))   
        model.add(layers.Dropout(hp.Float('dropoutdense_'+ str(i), 
                                      min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step)))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[ 1e-4])),
        loss='mean_squared_error',
        metrics=['mean_squared_error', 'mean_absolute_error'])
    return model


In [8]:
tuner = RandomSearch(
    build_model,
    objective='mean_squared_error',
    max_trials=4,
    project_name='hager')

INFO:tensorflow:Reloading Oracle from existing project .\hager\oracle.json
INFO:tensorflow:Reloading Tuner from .\hager\tuner0.json


In [9]:
arly_stopping = EarlyStopping(monitor='mean_squared_error', patience=30, verbose=1)
callback_list = [ early_stopping ]
tuner.search(X_train, y_train,
             epochs=60,
             batch_size=50, 
             callbacks=callback_list,
             validation_data=(X_test,y_test))
model = tuner.get_best_models(num_models=1)[0]


INFO:tensorflow:Oracle triggered exit

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001A7434A02E0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000001A744DC6610>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001A744DB7CD0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000001A744CEF280>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001A744D87DC0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000001A744CAAC40>).


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               39900     
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
flatten (Flatten)            (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 600)               180600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               120200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0

In [12]:
tuner.get_best_hyperparameters()[0].values

{'unit1': 300,
 'reg_rate': 0.01,
 'dropout_1': 0.6000000000000001,
 'num_layers': 2,
 'unitsdense_0': 600,
 'reg_rate1': 0.1,
 'dropoutdense_0': 0.4000000000000001,
 'learning_rate': 0.0001,
 'unitsdense_1': 200,
 'dropoutdense_1': 0.2}

In [13]:
best_model = tuner.get_best_models()[0]


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001A744CEFB20> and <tensorflow.python.keras.layers.core.Dropout object at 0x000001A744D619D0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001A744DB7910> and <tensorflow.python.keras.layers.core.Dropout object at 0x000001A744F3F280>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001A744E04C40> and <tensorflow.python.keras.layers.core.Dropout object at 0x000001A744E04C70>).


In [14]:
my_dict_Train={"RMSE": [],"MSE": [],"MBE": []};
my_dict_Test={"RMSE": [],"MSE": [],"MBE": []};

In [17]:
for i in range(0,2):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.10, stratify= y, shuffle=True, random_state=None)
        history= best_model.fit(X_train, y_train, epochs=15, batch_size=50,  verbose=1,shuffle=True, validation_data=(X_test,y_test))
        my_dict_Train["RMSE"].append(np.sqrt(np.mean(history.history['val_mean_squared_error'])) )
        my_dict_Train["MSE"].append(np.mean(history.history['val_mean_squared_error']))
        my_dict_Train["MBE"].append(np.mean(history.history['val_mean_absolute_error']))
        y_pred = best_model.predict(X_test)
        my_dict_Test["RMSE"].append(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
        my_dict_Test["MSE"].append(metrics.mean_squared_error(y_test,y_pred))
        my_dict_Test["MBE"].append(metrics.mean_absolute_error(y_test,y_pred))
        best_model.reset_states()

Epoch 1/15
12/12 [==============================] - 0s 18ms/step - loss: 5.7848 - mean_squared_error: 0.0644 - mean_absolute_error: 0.1624 - val_loss: 5.6071 - val_mean_squared_error: 0.0633 - val_mean_absolute_error: 0.1612
Epoch 2/15
12/12 [==============================] - 0s 11ms/step - loss: 5.4859 - mean_squared_error: 0.0648 - mean_absolute_error: 0.1625 - val_loss: 5.3164 - val_mean_squared_error: 0.0634 - val_mean_absolute_error: 0.1613
Epoch 3/15
12/12 [==============================] - 0s 12ms/step - loss: 5.2007 - mean_squared_error: 0.0644 - mean_absolute_error: 0.1624 - val_loss: 5.0396 - val_mean_squared_error: 0.0634 - val_mean_absolute_error: 0.1616
Epoch 4/15
12/12 [==============================] - 0s 11ms/step - loss: 4.9296 - mean_squared_error: 0.0645 - mean_absolute_error: 0.1628 - val_loss: 4.7762 - val_mean_squared_error: 0.0634 - val_mean_absolute_error: 0.1621
Epoch 5/15
12/12 [==============================] - 0s 12ms/step - loss: 4.6719 - mean_squared_error

In [18]:
print("Training result")
for metric in my_dict_Train.keys():
    print("%s: %.3f" % (metric, np.mean(my_dict_Train[metric])))  

Training result
RMSE: 0.252
MSE: 0.064
MBE: 0.165


In [19]:
print("Testing result")
for metric in my_dict_Test.keys():
    print("%s: %.3f" % (metric, np.mean(my_dict_Test[metric])))

Testing result
RMSE: 0.253
MSE: 0.064
MBE: 0.171
